# We explore this notebook and see how we can make modification to complete our tasks

Some ideas


*   Cross-gender identification
* Age identification
*   Deployment to mobile devices

Dataset provided in assignment brief: https://susanqq.github.io/UTKFace/

Download this: https://drive.google.com/drive/folders/0BxYys69jI14kU0I1YUQyY1ZDRUE?resourcekey=0-01Pth1hq20K4kuGVkp3oBw



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Age and Gender Estimation in TensorFlow ( Workbook 2, Gender Classification )**

* For Age Estimation, go to [Workbook 1 ( Age Estimation )](https://colab.research.google.com/drive/1to2iolQGIVZgXFWRmRKZGrny5kfXk40h?usp=sharing)

In this notebook, we train a Keras model to classify the gender of a person, given a *face-cropped* image. We use the famous [UTKFace Dataset](https://susanqq.github.io/UTKFace/), which contains 23K images where each image is labelled with its gender, age and ethinicity.


> **Note: Please make sure that you are connected to the GPU runtime of Google Colab. Else, the training might take a decade long. Go to Runtime > Change runtime type > Hardware accelerator.**

## 1) **Import libraries**

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from tensorflow.keras.layers import *
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, r2_score

## 2) **Processing the data**

Once we've downloaded the dataset, we need to perform the following operations on the dataset, so that it can be used for training our model,

* Reading the image files as 3D NumPy arrays. Note, we'll use 3-channeled RGB images for training the model, so each array will have a shape of `[ img_width , img_height , 3 ]`.

* Split the filename so as to parse the gender of the person in corresponding image. We use the `tf.strings.split()` method for performing this task.

* We one-hot encode the gender, as we'll perform *a two-class* classification.

Once this operations have been performed, we are left with $N$ samples where each sample consists of image array `[ 128 , 128 , 3 ]` and its corresponding label ( one-hot encoded ), the gender of that person, which has a shape `[ 1 , 2 ]`

We'll use `tf.data.Dataset` as it helps us to process the data faster, taking advantage of parallel computing. The above two operations will be mapped on each filename using `tf.data.Dataset.map` method.

In [2]:
# definitions for UTKDataset
dataset_dict = {
    'race_id': {
        0: 'white', 
        1: 'black', 
        2: 'asian', 
        3: 'indian', 
        4: 'others'
    },
    'gender_id': {
        0: 'male',
        1: 'female'
    }
}

dataset_dict['gender_alias'] = dict((g, i) for i, g in dataset_dict['gender_id'].items())
dataset_dict['race_alias'] = dict((r, i) for i, r in dataset_dict['race_id'].items())

In [3]:
# dir = '/content/drive/MyDrive/Dataset/UTK/UTKFace'
dir = './UTKDataset/UTKFace'

# Fraction of the dataset to be used for testing.
split = 0.3

image_data = []

def parse_data(filename):
  try:
    parts = filename.split('_')
    age = parts[0]
    gender = parts[1]
    race = parts[2]
    return int(age), dataset_dict['gender_id'][int(gender)], dataset_dict['race_id'][int(race)], filename
  except Exception as e:
    return None, None, None, None

for i in os.listdir(dir):
  age, gender, race, filename = parse_data(i)
  if age is not None and gender is not None and race is not None and filename is not None:
    image_data.append(parse_data(i))

data = pd.DataFrame(image_data, columns=["age", "gender", "race", "filename"])
data = data.sample(frac=1).reset_index(drop=True) # shuffle dataframe in place and reset index
data.head(10)


,age,gender,race,filename
0,29,female,black,29_1_1_20170115234825058.jpg.chip.jpg
1,80,female,white,80_1_0_20170110182107291.jpg.chip.jpg
2,38,male,asian,38_0_2_20170116192326283.jpg.chip.jpg
3,35,female,white,35_1_0_20170117135400203.jpg.chip.jpg
4,52,male,black,52_0_1_20170113174958212.jpg.chip.jpg
5,27,male,asian,27_0_2_20170119193330242.jpg.chip.jpg
6,26,male,white,26_0_0_20170117173443951.jpg.chip.jpg
7,24,female,indian,24_1_3_20170104215731414.jpg.chip.jpg
8,20,female,black,20_1_1_20170116164052068.jpg.chip.jpg
9,34,male,white,34_0_0_20170117121628625.jpg.chip.jpg


In [4]:
def plot_distribution(pd_series):
    labels = pd_series.value_counts().index.tolist()
    counts = pd_series.value_counts().values.tolist()
    
    pie_plot = go.Pie(labels=labels, values=counts, hole=.3)
    fig = go.Figure(data=[pie_plot])
    fig.update_layout(title_text='Distribution for %s' % pd_series.name)
    
    fig.show()

In [5]:
plot_distribution(data['gender'])
plot_distribution(data['race'])

In [ ]:
fig = px.histogram(data, x="age", nbins=20)
fig.update_layout(title_text='Age distribution on Train Dataset')
fig.show()

In [ ]:
plot_distribution(data['gender'])
plot_distribution(data['race'])

In [ ]:
fig = px.histogram(data, x="age", nbins=20)
fig.update_layout(title_text='Age distribution on Test Dataset')
fig.show()

Face dataset generator

In [4]:
img_height = 128
img_width = 128
split = 0.7 # train test split

class FaceDataGenerator():
  def __init__(self, dataframe):
    self.dataframe = dataframe

  def preprocess_image(self, file):
    img = Image.open(file)
    img = img.resize((img_width, img_height))
    img = np.array(img) / 255.0
    return img
  
  def alias(self):
    p = np.random.permutation(len(self.dataframe))
    train_up_to = int(len(self.dataframe) * split)
    train_idx = p[:train_up_to]
    test_idx = p[train_up_to:]
    train_up_to = int(train_up_to * split)
    train_idx, valid_idx = train_idx[:train_up_to], train_idx[train_up_to:]
    self.dataframe['gender_id'] = self.dataframe['gender'].map(lambda gender: dataset_dict['gender_alias'][gender.strip()])
    self.dataframe['race_id'] = self.dataframe['race'].map(lambda race: dataset_dict['race_alias'][race])
    self.max_age = self.dataframe['age'].max()

    return train_idx, valid_idx, test_idx

  def generate_images(self, i_idx, is_training, batch_size=8):
    # arrays to store our batched data
    images, ages, races, genders = [], [], [], []
    while True:
      for idx in i_idx:
          person = self.dataframe.iloc[idx]
          
          age = person['age']
          race = person['race_id']
          gender = person['gender_id']
          filename = person['filename']
          
          im = self.preprocess_image(dir + '/' + filename)
          
          ages.append(age / self.max_age)
          races.append(to_categorical(race, len(dataset_dict['race_id'])))
          genders.append(to_categorical(gender, len(dataset_dict['gender_id'])))
          images.append(im)
          
          # yielding condition
          if len(images) >= batch_size:
              yield np.array(images), [np.array(ages), np.array(races), np.array(genders)]
              images, ages, races, genders = [], [], [], []
              
      if not is_training:
          break

In [5]:
generator = FaceDataGenerator(data)
train_idx, valid_idx, test_idx = generator.alias()

In [6]:
print(f'Training sample: {len(train_idx)}')
print(f'Validation sample: {len(valid_idx)}')
print(f'Testing sample: {len(test_idx)}')

Training sample: 11615
Validation sample: 4978
Testing sample: 7112



## 3) Model

Our aim is to develop a model which has lesser parameters ( which implies lesser inference time and size ) but powerful enough so that it can generalize better.

* The model takes in a batch of shape `[ None , 128 , 128 , 3 ]` and performs a number of convolutions on it as determined by `num_blocks`.
* Each block consists of a sequence of layers : `Conv2D -> BatchNorm -> LeakyReLU`



* If `lite_model` is set to `True`, we use [Separable Convolutions](https://towardsdatascience.com/a-basic-introduction-to-separable-convolutions-b99ec3102728) which have lesser parameters. We could achieve a *faster* model, compromising its performance.

* We stack such`num_blocks` blocks sequentially, where the no. of filters for each layer is taken from `num_filters`.

* Next we add a number of `Dense` layers to learn the features extracted by convolutional layers. Note, we also add a `Dropout` layer, to reduce overfitting. The `rate` for each `Dropout` layer is decreased subsequently for each layer, so that the learnability of `Dense` layer with lesser units ( neurons ) is not affected.

* The last `Dense` layer applies the softmax activation function which yields a probability distribution for the two classes `male` and `female`.

> See [this](https://machinelearningmastery.com/how-to-reduce-overfitting-in-deep-learning-with-weight-regularization/) blog for choosing the weight decay values used in the above two blocks.

* 👉🏻 The output of the model is a tensor with shape `[ None, 2 ]`


In [7]:
MODEL_INPUT_IMAGE_SIZE = [128, 128]
# Negative slope coefficient for LeakyReLU.
leaky_relu_alpha = 0.2

lite_model = True

# Define the conv block.
def conv( x , num_filters , kernel_size=( 3 , 3 ) , strides=1 ):
    if lite_model:
        x = tf.keras.layers.SeparableConv2D( num_filters ,
                                            kernel_size=kernel_size ,
                                            strides=strides, 
                                            use_bias=False ,
                                            kernel_initializer=tf.keras.initializers.HeNormal() ,
                                            kernel_regularizer=tf.keras.regularizers.L2( 1e-5 )
                                             )( x )
    else:
        x = tf.keras.layers.Conv2D( num_filters ,
                                   kernel_size=kernel_size ,
                                   strides=strides ,
                                   use_bias=False ,
                                   kernel_initializer=tf.keras.initializers.HeNormal() ,
                                   kernel_regularizer=tf.keras.regularizers.L2( 1e-5 )
                                    )( x )

    x = tf.keras.layers.BatchNormalization()( x )
    x = tf.keras.layers.LeakyReLU( leaky_relu_alpha )( x )
    return x

def dense( x , filters , dropout_rate ):
    x = tf.keras.layers.Dense( filters , kernel_regularizer=tf.keras.regularizers.L2( 0.1 ) , bias_regularizer=tf.keras.regularizers.L2( 0.1 ) )( x )
    x = tf.keras.layers.LeakyReLU( alpha=leaky_relu_alpha )( x )
    x = tf.keras.layers.Dropout( dropout_rate )( x )
    return x

# No. of convolution layers to be added.
num_blocks = 5

# Num filters for each conv layer.
num_filters = [ 16 , 32 , 64 , 128 , 256 , 256 ]

# Kernel sizes for each conv layer.
kernel_sizes = [ 3 , 3 , 3 , 3 , 3 , 3 ]

# Init a Input Layer.
inputs = tf.keras.layers.Input(shape=MODEL_INPUT_IMAGE_SIZE + [3])

def standard_conv_layers(inputs):
  x = Conv2D(16, (3, 3), padding="same")(inputs)
  x = Activation("relu")(x)
  x = BatchNormalization(axis=-1)(x)
  x = MaxPooling2D(pool_size=(3, 3))(x)
  x = Dropout(0.25)(x)

  x = Conv2D(32, (3, 3), padding="same")(x)
  x = Activation("relu")(x)
  x = BatchNormalization(axis=-1)(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)
  x = Dropout(0.25)(x)

  x = Conv2D(32, (3, 3), padding="same")(x)
  x = Activation("relu")(x)
  x = BatchNormalization(axis=-1)(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)
  x = Dropout(0.25)(x)

  return x
#################### Gender Classifier ###########################
# Flatten the output of the last Conv layer.
x = inputs
# x = Lambda(lambda c: tf.image.rgb_to_grayscale(c))(inputs)
x = standard_conv_layers(x)
x = Flatten()(x)
x = Dense(128)(x)
x = Activation("relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
gender = Dense(2, activation='sigmoid', name='gender_output')(x)
#################### End of Gender Classifier ###########################

#################### Race Classifier ###########################
x = inputs
x = standard_conv_layers(inputs)
x = Flatten()(x)
x = Dense(128)(x)
x = Activation("relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
race = tf.keras.layers.Dense(5, activation='softmax', name='race_output')(x)
#################### End of Race Classifier ###########################

#################### Age Classifier ###########################
x = inputs
x = standard_conv_layers(inputs)
x = Flatten()(x)
x = Dense(128)(x)
x = Activation("relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
age = tf.keras.layers.Dense(1, activation='linear', name='age_output')(x)
#################### End of Age Classifier ###########################

# Build the Model
model = tf.keras.models.Model(inputs , [age, race, gender])

# Uncomment the below to view the summary of the model.
model.summary()
# tf.keras.utils.plot_model(model) # add to_file='architecture.png' to export architecture 


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 128, 128, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 128, 128, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 16) 448         input_1[0][0]                    
______________________________________________________________________________________________


Run this cell to visualize the training of the model in TensorBoard ( in this notebook itself ).

## 4) **Compiling the model ( and other callbacks )** 🧱

Once we've defined the architecture for our model, we'll compile our Keras model and also initialize some useful callbacks.

* As we're performing classification, we'll use the Categorical Crossentropy loss function. See [`tf.keras.losses.CategoricalCrossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy) for more details.

* We'll use the Adam optimizer for training our model. See [`tf.keras.optimizers.Adam`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam) for more details.

* For evaluating the performance of our model, we measure the accuracy of our model. See [`tf.keras.metrics.Accuracy`](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/Accuracy) for more details.


#### Callbacks:

* [`tf.keras.callbacks.ModelCheckpoint`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint) to save the Keras model as an H5 file after every epoch.

* [`tf.keras.callbacks.TensorBoard`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard) to visualize the training with TensorBoard.

* [`tf.keras.callbacks.EarlyStopping`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping) to stop the training when the evaluation metric i.e the MAE stops improving on the test dataset.

In [8]:
learning_rate = 1e-4
num_epochs = 50 

save_dir = './checkpoints'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(save_dir)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=25)
logdir = os.path.join("./logs/tb_logs" , datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.compile(
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=learning_rate,
        decay=learning_rate/num_epochs
    ),
    loss = {
        'age_output': 'mse',
        'race_output': 'categorical_crossentropy',
        'gender_output': 'binary_crossentropy'
    },
    loss_weights={
        'age_output': 4., 
        'race_output': 1.5, 
        'gender_output': 0.1
    },
    metrics = {
        'age_output': 'mae',
        'race_output': 'accuracy',
        'gender_output': 'accuracy'
    }
)


## 5) **Train and Evaluate the Model** 🏋🏻‍♂️

Start the training loop with all callbacks packed in.


In [9]:
batch_size = 32

train_data = generator.generate_images(train_idx, is_training = True, batch_size = batch_size)
test_data = generator.generate_images(valid_idx, is_training = True, batch_size = batch_size)

# we use fit_generator here because data augmentation is performed here
history = model.fit_generator( 
    train_data,
    steps_per_epoch=len(train_idx)//batch_size,
    epochs=num_epochs,
    validation_data=test_data,
    validation_steps=len(valid_idx)//batch_size,
    verbose=1,
    callbacks=[checkpoint_callback , tensorboard_callback , early_stopping_callback]
)

D:\anaconda3\envs\MDP\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
362/362 [==============================] - 40s 101ms/step - loss: 14.7739 - age_output_loss: 2.9119 - race_output_loss: 2.0329 - gender_output_loss: 0.7712 - age_output_mae: 1.3343 - race_output_accuracy: 0.3625 - gender_output_accuracy: 0.6823 - val_loss: 8.3442 - val_age_output_loss: 1.1577 - val_race_output_loss: 2.4156 - val_gender_output_loss: 0.9016 - val_age_output_mae: 0.8843 - val_race_output_accuracy: 0.4218 - val_gender_output_accuracy: 0.5377
INFO:tensorflow:Assets written to: .\checkpoints\assets
Epoch 2/50
362/362 [==============================] - 29s 80ms/step - loss: 9.9246 - age_output_loss: 1.8618 - race_output_loss: 1.6121 - gender_output_loss: 0.5940 - age_output_mae: 1.0683 - race_output_accuracy: 0.4540 - gender_output_accuracy: 0.7599 - val_loss: 8.8957 - val_age_output_loss: 1.4783 - val_race_output_loss: 1.9486 - val_gender_output_loss: 0.5953 - val_age_output_mae: 0.6638 - val_race_output_accuracy: 0.4369 - val_gender_output_accuracy: 0.7546
INFO:t

## Evaluate the Model.

In [10]:
# Overall Loss
fig = go.Figure()
fig.add_trace(go.Scattergl(
                    y=history.history['loss'],
                    name='Train'))

fig.add_trace(go.Scattergl(
                    y=history.history['val_loss'],
                    name='Valid'))


fig.update_layout(height=500, 
                  width=700,
                  title='Overall loss',
                  xaxis_title='Epoch',
                  yaxis_title='Loss')

fig.show()

Save the Keras model to the local disk, so that we can resume training if needed.

## Test

In [11]:
test_batch_size = 128
test_generator = generator.generate_images(test_idx, is_training=False, batch_size=test_batch_size)
age_pred, race_pred, gender_pred = model.predict(test_generator, steps=len(test_idx)//test_batch_size)


In [12]:
test_generator = generator.generate_images(test_idx, is_training=False, batch_size=test_batch_size)
samples = 0
images, age_true, race_true, gender_true = [], [], [], []
for test_batch in test_generator:
    image = test_batch[0]
    labels = test_batch[1]
    
    images.extend(image)
    age_true.extend(labels[0])
    race_true.extend(labels[1])
    gender_true.extend(labels[2])
    
age_true = np.array(age_true)
race_true = np.array(race_true)
gender_true = np.array(gender_true)

race_true, gender_true = race_true.argmax(axis=-1), gender_true.argmax(axis=-1)
race_pred, gender_pred = race_pred.argmax(axis=-1), gender_pred.argmax(axis=-1)

age_true = age_true * generator.max_age
age_pred = age_pred * generator.max_age


In [13]:
cr_race = classification_report(race_true, race_pred, target_names=dataset_dict['race_alias'].keys())
print(cr_race)

              precision    recall  f1-score   support

       white       0.73      0.95      0.83      2949
       black       0.89      0.77      0.82      1371
       asian       0.89      0.74      0.81      1037
      indian       0.75      0.69      0.72      1168
      others       0.39      0.08      0.13       515

    accuracy                           0.78      7040
   macro avg       0.73      0.64      0.66      7040
weighted avg       0.76      0.78      0.75      7040



In [14]:
cr_gender = classification_report(gender_true, gender_pred, target_names=dataset_dict['gender_alias'].keys())
print(cr_gender)

              precision    recall  f1-score   support

        male       0.95      0.82      0.88      3681
      female       0.83      0.96      0.89      3359

    accuracy                           0.89      7040
   macro avg       0.89      0.89      0.89      7040
weighted avg       0.89      0.89      0.89      7040



In [15]:
print('R2 score for age: ', r2_score(age_true, age_pred))

R2 score for age:  0.6908394168881167


In [ ]:
fig = plt.figure(figsize=(128, 128 ))
rows = 15
columns = 1

i = 0
for image , age_p, race_p, gender_p, age, race, gender in zip(images, age_pred, race_pred, gender_pred, age_true, race_true, gender_true):
    i += 1
    image = np.array(image)
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(image)
    plt.axis('off')
    # plt.title( 'Predicted gender : {} , actual gender : {}'.format( label_ , classes[ np.argmax( label ) ] ) )
    plt.title(f'Predicted Age: {age_p} Actual Age: {age} \n Predicted Gender: {dataset_dict["gender_id"][gender_p]} Actual Gender: {dataset_dict["gender_id"][gender]} \n Predicted Race: {dataset_dict["race_id"][race_p]} Actual Race: {dataset_dict["race_id"][race]}')


 
## 7) **Convert to TensorFlow Lite format**

Our model is to be deployed in an Android app, where we'll use [TF Lite Android](https://bintray.com/google/tensorflow/tensorflow-lite) package to parse the model and make predictions.

We use the `TFLiteConverter` API to convert our Keras Model ( `.h5` ) to a TF Lite buffer ( `.tflite` ). See the [official docs](https://www.tensorflow.org/api_docs/python/tf/lite/TFLiteConverter/). We'll produce two TF Lite buffers, one with float16 quantization and other non-quantized model.


In [ ]:

converter = tf.lite.TFLiteConverter.from_keras_model( model )
converter.optimizations = [ tf.lite.Optimize.DEFAULT ]
converter.target_spec.supported_types = [ tf.float16 ]
buffer = converter.convert()

open( '{}_q.tflite'.format( model_name ) , 'wb' ).write( buffer )
files.download( '{}_q.tflite'.format( model_name ) )



For conversion to a non-quantized TF Lite buffer.


In [ ]:

converter = tf.lite.TFLiteConverter.from_keras_model( model )
buffer = converter.convert()

open( '{}_nonq.tflite'.format( model_name ) , 'wb' ).write( buffer )
files.download( '{}_nonq.tflite'.format( model_name ) )



## Utility Methods

Use these methods to automate some of the tasks.


In [ ]:

#@title Utility to zip and download a directory
#@markdown Use this method to zip and download a directory. For ex. a TB logs 
#@markdown directory or a checkpoint(s) directory.

dir_to_zip = 'tb_logs' #@param {type: "string"}
output_filename = 'logs.zip' #@param {type: "string"}
delete_dir_after_download = "No"  #@param ['Yes', 'No']

os.system( "zip -r {} {}".format( output_filename , dir_to_zip ) )

if delete_dir_after_download == "Yes":
    os.system( "rm -r {}".format( dir_to_zip ) )

files.download( output_filename )


In [ ]:

#@title Utility to delete a directory
#@markdown Use this method to delete a directory. 

dir_path = ''  #@param {type: "string"}
os.system( f'rm -r {dir_path}')


In [ ]:
%load_ext tensorboard
%tensorboard --logdir tb_logs/